<a href="https://colab.research.google.com/github/Informathiques/2024-Autumn-Cluj-Napoca/blob/main/Day1_dataEntryForm_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation, please run it

In [ ]:
# @title
!pip install pymongo[srv] > /dev/null 2>&1

from google.colab import userdata
import os
import base64

# Base64-encoded MongoDB URI (fallback option)
encoded_uri = "bW9uZ29kYitzcnY6Ly9pbmZyb21hdGhpcXVlczpkRk91WTZYaWFRNjhxeE5KQGluZm9ybWF0aGlxdWVzLjgzanFlLm1vbmdvZGIubmV0Lz9yZXRyeVdyaXRlcz10cnVlJnc9bWFqb3JpdHkmc3NsPXRydWUmdGxzQWxsb3dJbnZhbGlkQ2VydGlmaWNhdGVzPXRydWU="

# Step 1: Try to get the MongoDB URI from the environment variable (SMONGO_URI)
try:
    MONGO_URI = os.getenv("SMONGO_URI")
    if not MONGO_URI:
        raise ValueError("SMONGO_URI not found or is empty.")
except (KeyError, ValueError, Exception) as e:
    #print(f"Environment variable 'SMONGO_URI' is missing or cannot be accessed: {e}")
    #print("Falling back to the base64-encoded URI...")
    # Step 2: Decode the base64-encoded URI as fallback
    MONGO_URI = base64.b64decode(encoded_uri).decode('utf-8')
print("Succes on initialisation. Continue")



Succes on initialisation. Continue


# This part of the code will ask you to enter some data about you

In [ ]:
# @title
import pymongo
from pymongo import MongoClient
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

client = MongoClient(MONGO_URI)
db = client["myDatabase"]  # Database name
collection = db["users"]  # Collection name

# Global variable to store current user ID for editing
current_user_id = None

# Function to add or update user in MongoDB
def add_or_update_user(first_name, second_name, height, gender, dob):
    global current_user_id
    # Convert the Date of Birth to datetime
    dob_datetime = datetime.combine(dob, datetime.min.time()) if dob else None

    user_data = {
        "First Name": first_name,
        "Second Name": second_name,
        "Height": height,
        "Gender": gender,
        "Date of Birth": dob_datetime
    }

    if current_user_id is None:
        # Insert new user
        result = collection.insert_one(user_data)
        current_user_id = result.inserted_id
        with message_output:
            clear_output()
            print(f"User {first_name} {second_name} added to the database.")
    else:
        # Update existing user
        collection.update_one({"_id": current_user_id}, {"$set": user_data})
        with message_output:
            clear_output()
            print(f"User {first_name} {second_name}'s data updated in the database.")

# Function to display an error message (acts like a modal)
def show_error(message):
    with message_output:
        clear_output()
        print(f"Error: {message}")

# Function to handle the submit action
def on_submit(b):
    global current_user_id
    first_name = first_name_input.value
    second_name = second_name_input.value
    height = int(height_input.value)
    gender = gender_input.value
    dob = dob_input.value

    # Validation: Check if first name, second name, and date of birth are filled
    if not first_name or not second_name:
        show_error("First Name and Second Name cannot be empty.")
        return

    if not dob:
        show_error("Date of Birth cannot be empty.")
        return

    # Insert or update the user data in MongoDB
    add_or_update_user(first_name, second_name, height, gender, dob)

    # Disable fields and activate edit button
    first_name_input.disabled = True
    second_name_input.disabled = True
    height_input.disabled = True
    gender_input.disabled = True
    dob_input.disabled = True
    submit_button.disabled = True
    edit_button.disabled = False

# Function to handle the edit action
def on_edit(b):
    # Enable fields and disable edit button
    first_name_input.disabled = False
    second_name_input.disabled = False
    height_input.disabled = False
    gender_input.disabled = False
    dob_input.disabled = False
    submit_button.disabled = False
    edit_button.disabled = True

# Customizing font size and layout
text_layout = widgets.Layout(width='300px', height='30px', font_size='16px')
button_style = widgets.ButtonStyle(button_color='lightblue')

# Create input fields with initial disabled state for edit button
first_name_input = widgets.Text(description="First Name  ", layout=text_layout)
second_name_input = widgets.Text(description="Second Name", layout=text_layout)
height_input = widgets.BoundedFloatText(value=150, min=100, max=300, description="Height (cm)", layout=text_layout)

# Gender radio buttons (now stacked vertically using VBox)
gender_input = widgets.RadioButtons(
    options=['Male', 'Female'],
    value='Male',  # Default value
    description='Gender',
    layout=widgets.Layout(width='auto')  # Allow auto width to fit both options
)

# Date of Birth input
dob_input = widgets.DatePicker(description='Date of Birth', layout=text_layout)

# Submit and Edit buttons with custom color and font size
submit_button = widgets.Button(description="Submit", style=button_style, layout=widgets.Layout(width='150px', height='40px', font_size='16px'))
edit_button = widgets.Button(description="Edit", disabled=True, style=button_style, layout=widgets.Layout(width='150px', height='40px', font_size='16px'))

# Output widget to display error or success messages
message_output = widgets.Output()

# Attach button event handlers
submit_button.on_click(on_submit)
edit_button.on_click(on_edit)

# Group gender input and date picker into an HBox for better layout
gender_dob_box = widgets.HBox([gender_input, dob_input])

# Display the widgets using VBox for vertical alignment
display(widgets.VBox([first_name_input, second_name_input, height_input, gender_dob_box, submit_button, edit_button, message_output]))
